In [1]:
import os
import numpy as np
from tsnecuda import TSNE
from tqdm.auto import tqdm
from cache_decorator import Cache
from sklearn.preprocessing import StandardScaler
from repairing_genomic_gaps import build_multivariate_dataset_cae, get_model_weights_path
from repairing_genomic_gaps import cae_1000

# Latent space rendering

In [2]:
@Cache()
def build_latent(model):
    # Load the model
    model = globals()[model]
    auto, enc, dec = model(return_encoder_and_decoder=True, verbose=False)
    
    # Load the weights
    path = get_model_weights_path(auto, "./multivariate_gaps")
    if os.path.exists(path):
        auto.load_weights(path)
        print("Old Weights loaded from {}".format(path))
    ## Already tested that if we load the weights for auto
    ## the weights changes also for enc and dec
    
    # Load the dataset
    train, test = build_multivariate_dataset_cae(1000)
    # Render the latent space
    latent = np.array([
        enc.predict(point)
        for batch in tqdm(train, desc="Rendering the latent space")
        for point in batch
    ])
    return latent

In [ ]:
latent = build_latent("cae_1000")

# Latent space normalizzation

In [ ]:
normalized_latent = StandardScaler().fit_transform(latent)

# TSNE rendering

In [3]:
@Cache()
def build_tsne(latent, n_components=2):
    embeddedX = TSNE(n_components=n_components).fit_transform(latent)
    return embeddedX

In [ ]:
t = build_tsne(normalized_latent)

Old Weights loaded from ./multivariate_gaps/cae_1000/best_weights_cae_1000.hdf5


# TSNE visualizzation

# PCA rendering

# PCA visualizzation